In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import np_utils
from keras.layers import Dense, Activation,Flatten, Convolution2D, Dropout, MaxPooling2D
from keras.models import Sequential

import datetime

In [22]:
ds = pd.read_csv('d:/ML--perceptron/mnist/train.csv')
data = ds.values[:12000]
print (data.shape)

(12000, 785)


In [23]:
## Model1 -> (0-4) and model 2 -> (5-9)
mask = data[:,0] < 5
print (data[:,0].shape)
print (mask.shape)
print (np.unique(mask))
print (mask.sum())

data_01 = []
data_02 = []

for ix in range(data.shape[0]):
    if mask[ix] == True:
        data_01.append(data[ix])
    else:
        data_02.append(data[ix])

data_01 = np.asarray(data_01)
data_02 = np.asarray(data_02)
print ("---")
print (data_01.shape)
print (data_02.shape)

(12000,)
(12000,)
[False  True]
6134
---
(6134, 785)
(5866, 785)


In [31]:
## data_01 for first network: 
split = int(0.85*data_01.shape[0])

x_tr = data_01[:split, 1:]/255.0
x_tes = data_01[split:, 1:]/255.0

y_tr = np_utils.to_categorical(data_01[:split,0], nb_classes=5)
y_tes = np_utils.to_categorical(data_01[split:, 0], nb_classes=5)

x_tr = x_tr.reshape((-1,28,28,1))
x_tes = x_tes.reshape((-1,28,28,1))

print (x_tr.shape, x_tes.shape)
print (y_tr.shape, y_tes.shape)

(5213, 28, 28, 1) (921, 28, 28, 1)
(5213, 5) (921, 5)


In [32]:
model = Sequential()

model.add(Convolution2D(64,3,3, input_shape = (28,28,1), activation='relu'))
model.add(Convolution2D(32,3,3, activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(16,3,3, activation='relu'))
model.add(Convolution2D(8,3,3, activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_13 (Convolution2D) (None, 26, 26, 64)    640         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 24, 24, 32)    18464       convolution2d_13[0][0]           
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 12, 12, 32)    0           convolution2d_14[0][0]           
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 10, 10, 16)    4624        maxpooling2d_4[0][0]             
___________________________________________________________________________________________

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
start = datetime.datetime.now()
ans = start - datetime.datetime.now()
print (ans)

0:00:00


In [36]:
hist = model.fit(x_tr,y_tr,batch_size=16, nb_epoch=5, verbose=2, validation_data=(x_tes,y_tes))

Train on 5213 samples, validate on 921 samples
Epoch 1/5
23s - loss: 0.1523 - acc: 0.9555 - val_loss: 0.0632 - val_acc: 0.9783
Epoch 2/5
27s - loss: 0.0881 - acc: 0.9762 - val_loss: 0.0425 - val_acc: 0.9805
Epoch 3/5
25s - loss: 0.0611 - acc: 0.9824 - val_loss: 0.0364 - val_acc: 0.9859
Epoch 4/5
22s - loss: 0.0389 - acc: 0.9875 - val_loss: 0.0501 - val_acc: 0.9848
Epoch 5/5
22s - loss: 0.0359 - acc: 0.9893 - val_loss: 0.0409 - val_acc: 0.9848


In [50]:
transfer_model = Sequential(model.layers[:-4])

for ix in transfer_model.layers:
    ix.trainable = False

#transfer_model.add(Flatten)
#transfer_model.add(Dense(100))
#transfer_model.add(Dropout(0.5))
#transfer_model.add(Activation('relu'))



transfer_model.add(Dense(5))
transfer_model.add(Activation('softmax'))
transfer_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_13 (Convolution2D) (None, 26, 26, 64)    640         convolution2d_input_4[0][0]      
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 24, 24, 32)    18464       convolution2d_13[0][0]           
                                                                   convolution2d_13[0][0]           
                                                                   convolution2d_13[0][0]           
                                                                   convolution2d_13[0][0]           
                                                                   convolution2d_13[0][0]           
                                                                   convolution2d_13[0][0]  

In [70]:
split = int(0.85 * data_02.shape[0])

X_tr = data_02[:split,1:]/255.0
X_te = data_02[split:,1:]/255.0

X_tr = X_tr.reshape((-1,28,28,1))
X_te = X_te.reshape((-1,28,28,1))

y_tr = np_utils.to_categorical(data_02[:split,0]-5,nb_classes=5)
y_te = np_utils.to_categorical(data_02[split:,0]-5,nb_classes=5)

print(X_tr.shape, X_te.shape)
print(y_tr.shape, y_te.shape)

(4986, 28, 28, 1) (880, 28, 28, 1)
(4986, 5) (880, 5)


In [71]:
transfer_model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [73]:
start = datetime.datetime.now()
hist = transfer_model.fit(X_tr,y_tr,nb_epoch=10,shuffle=True,batch_size=128,validation_data=(X_te,y_te))
print (datetime.datetime.now()-start)

Train on 4986 samples, validate on 880 samples
Epoch 1/10
4986/4986 [==============================] - 8s - loss: 0.1315 - acc: 0.9545 - val_loss: 0.1419 - val_acc: 0.9568
Epoch 2/10
4986/4986 [==============================] - 8s - loss: 0.1294 - acc: 0.9583 - val_loss: 0.1416 - val_acc: 0.9591
Epoch 3/10
4986/4986 [==============================] - 8s - loss: 0.1284 - acc: 0.9585 - val_loss: 0.1402 - val_acc: 0.9580
Epoch 4/10
4986/4986 [==============================] - 8s - loss: 0.1266 - acc: 0.9597 - val_loss: 0.1380 - val_acc: 0.9591
Epoch 5/10
4986/4986 [==============================] - 8s - loss: 0.1253 - acc: 0.9591 - val_loss: 0.1362 - val_acc: 0.9580
Epoch 6/10
4986/4986 [==============================] - 9s - loss: 0.1238 - acc: 0.9577 - val_loss: 0.1363 - val_acc: 0.9580
Epoch 7/10
4986/4986 [==============================] - 9s - loss: 0.1223 - acc: 0.9601 - val_loss: 0.1363 - val_acc: 0.9580
Epoch 8/10
4986/4986 [==============================] - 8s - loss: 0.1214 - ac